# RNN from scratch, only using numpy
#### By: Teddy Ordoñez
##### Source: https://pythonalgos.com/build-a-recurrent-neural-network-from-scratch-in-python-3/

In [6]:
# imports
import math
import numpy as np
import matplotlib.pyplot as plt

#### Defining RNN architecture

The RNN will have a learning rate of 0.001, the sequence length of the sin wave is 50, the max number os epochs will be 25, the hidden dimension will have a size of 100, output dimension will have a size of 1, backpropagating the error for every sequence, with a max and min values of 10 and -10, respectively

In [10]:
learning_rate = 0.0001
sequence_length = 50
epochs_max = 25
hidden_dimension = 100
output_dimension = 1
bptt = 5   # backpropagating the error, change value to sequence_length (50)
min_clip_val = -10
max_clip_val = 10

#### Activation Function

For this RNN, we will be using Sigmoid Function as our activation function

In [9]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

#### Loss Calculation

Creating a Loss Calculation function. Which will receive input (**X**) and result (**Y**) matrices, input to hidden layer weigths (**U**), hidden to output layer weigths (**V**) and hidden-to-hidden weigths (**W**). 

In [11]:
# Where X = data matrix, Y = output matrix, U = input to hidden weigths, V = hidden to output weights and W = hidden to hidden weights
def loss_calculation(X, Y, U, V, W):
    loss = 0.0
    
    for i in range(Y.shape[0]):
        x, y = X[i], Y[i]   # x and y will represent a specific data point
        previous_activation = np.zeros((hidden_dimension, 1))   # previous activation needs to have the same size as the hidden dimension

        for timestep in range(sequence_length): # Sequence length determines the timestep
            new_input = np.zeros(x.shape)   # New input will hold every data point in x with the shape of x. Doing this for every step in the sequence, forwards pass.
            new_input[timestep] = x[timestep]   # New input now has the same value os data entry for that timestep. New input has a single input for that timestep
            multiplied_u = np.dot(U, new_input) # Multipliying the inputs times the weights
            multiplied_w = np.dot(W, previous_activation)   # Multiplying the previous activation values times the hidden-to-hidden weigths
            sum_mulu_mulw = multiplied_u + multiplied_w     # Suming the products of the inputs and activations with their respective weigths
            new_activation = sigmoid(sum_mulu_mulw)     # Activating that sum
            multiplied_v = np.dot(V, new_activation)    # Multiplying the activated values time the weights of hidden-to-output layer
            previous_activation = new_activation        # The current activations becomes the previous activation for the next iteration
        
        loss_per_input = float((y - multiplied_v) ** 2 / 2)     # Calculating the Mean Squared Error (MSE)
        loss += loss_per_input      # Adding the input loss to the total loss

    return loss, new_activation     # Returning the total loss and activation values

#### Layer Activation Calculation

With this function we will be calculating the activation values of the recurrent layers created by the recurrance relation of the RNN. This function receives input matrix (**x**), input to hidden weights (**U**), hidden to output weights (**V**), hidden to hidden weights (**W**) and the previous activation values(**previous_activation**)

In [12]:
# Where X = data matrix, U = input to hidden weigths, V = hidden to output weights and W = hidden to hidden weights and previous_activation = previous activation for the final layer
def layer_activation_calc(x, U, V, W, previous_activation):
    layers = []     # Creating a list of empty layers before iterating for each timestep
    for timestep in range(sequence_length):
        new_input = np.zeros(x.shape)   # New input will begin with 0 in a x-like shape 
        new_input[timestep] = x[timestep]  # New input now has the same value os data entry for that timestep. New input has a single input for that timestep
        multiplied_u = np.dot(U, new_input)     # Multiplying inputs times their weights in relation to hidden layer
        multiplied_w = np.dot(V, previous_activation)    # Multipliying previous activation times hidden to output layer weights
        sum_mulu_mulw = multiplied_u + multiplied_w      # Adding both results
        activation = sigmoid(sum_mulu_mulw)     # Activating result
        multiplied_v = np.dot(V, activation)    # Multiplying the activated results times hidden to output weights
        layers.append({'activation' : activation, 'previous_activation' : previous_activation})     # Creating a dictionary containing the new activation and previous activation values
        previous_activation = activation    # Updating previous activation with the new activation

    return layers, multiplied_u, multiplied_w, multiplied_v     # Returning the recurrent layers, and multiplied U, W, and V matrices

#### Previous Hidden Layer Activation Differential

Since finding the differntial for the previous activation of the hidden layer needs to be performed several times, creating a function will help to keep the code cleaner. 
The fuction receives three parameters, first we need the sum of the weights (**sum_weigths**), the differential of the output layer (**ds**) and finally the weigths layer (**W**)

In [13]:
def previous_activation_differential(sum_weights, ds, W):
    d_sum_weights = sum_weights * (1 - sum_weights) * ds    # Calculating differential by mult the sum of the weights by its "inv" from 1, times the differential of the output layer
    d_mul_w = d_sum_weights * np.ones_like(ds)  # Differential of the hidden layerr output by multiplying diff of the sum by a matrix in the shape of the output layer diff
    return np.dot(np.transpose(W), d_mul_w)     # Returning the dot product of the hidden layer weigth and the differential created before

#### Backpropagation

We will use backpropagation to update the weights of our neural network. The function will receive eight parameters (all have been created before). **x** is the input matrix, **U** is input to hidden weigths, **V** is hidden to output weigths, **W** is hidden to hidden weights, **d_multiplied_v** is the differential for the last layer, **multiplied_u** and **multiplied_w** are the input values for the hidden layer, and finally **layers** is the list of layer activaitons

In [14]:
# Where X = data matrix, U = input to hidden weigths, V = hidden to output weights and W = hidden to hidden weights, d_multiplied_v = differential last layer, 
# multiplied_u = multiplied input to hidden values, multiplied_w = multiplied hidden to hidden values, and layers = layer activations
def backpropagation(x, U, V, W, d_multiplied_v, multiplied_u, multiplied_w, layers):
    dU = np.zeros(U.shape)      # These three lines prepare the variables that will hold the differential of U, V, and W
    dV = np.zeros(V.shape)
    dW = np.zeros(W.shape)
    dU_t = np.zeros(U.shape)      # These three lines prepare the variables that will hold the differential in the timestep of U, V, and W
    dV_t = np.zeros(V.shape)
    dW_t = np.zeros(W.shape)
    dU_i = np.zeros(U.shape)      # These two lines will hold the differentials of the backprop through time
    dW_i = np.zeros(W.shape)
    sum_mulu_mulw = multiplied_u + multiplied_w     # Adding input to hidden weigths and hidden to hidden weights
    dsv = np.dot(np.transpose(V), d_multiplied_v)   # Multiplying the transposedd matrix of V times the differential of V

    for timestep in range(sequence_length):
        dV_t  = np.dot(d_multiplied_v, np.transpose(layers[timestep]['activation'])) # Diff = last layer differential times last layer activation
        ds = dsv    # Copying the differential of the last layer into ds, since we are going to change it for this timestep
        d_previous_activation = previous_activation_differential(sum_mulu_mulw, ds, W)  # Getting differential of previous activation

        for x in range(timestep - 1, max(-1, timestep - bptt - 1), -1): # Backprop by looping through each prior timestep
            ds = dsv + d_previous_activation    # Agumenting the last layer differential
            d_previous_activation = previous_activation_differential(sum_mulu_mulw, ds, W)  # Calculating the previous activation in the previous timestep with the new differential
            dW_i = np.dot(W, layers[timestep]['previous_activation'])   # Calculating the differential for this recurrent timestep by mult hidden weigths time timestep's prev activation

            new_input = np.zeros(x.shape)   
            new_input[timestep] = x[timestep]   # New input is the current timestep
            dU_i = np.dot(U, new_input)     # Differential for te input layer for this recurrent timestep

            dU_t += dU_i        # Adding differential input and hidden values of the recurrent timestep
            dW_t += dW_i
        
        dU += dU_t      # Adding the differentials of each timestep to store them
        dV += dV_t
        dW += dW_t

        # Exploding gradients, if any differential is bigger or smalled than selected boundaries, change value to the boundary to avoid exploding gradients
        if dU.max() > max_clip_val:
            dU[dU > max_clip_val] = max_clip_val
        if dV.max() > max_clip_val:
            dV[dV > max_clip_val] = max_clip_val
        if dW.max() > max_clip_val:
            dW[dW > max_clip_val] = max_clip_val
       
        if dU.min() < min_clip_val:
            dU[dU < min_clip_val] = min_clip_val
        if dV.min() < min_clip_val:
            dV[dV < min_clip_val] = min_clip_val
        if dW.min() < min_clip_val:
            dW[dW < min_clip_val] = min_clip_val
    
    return dU, dV, dW